# AI Community @ Семинар № 7, весна 2019
## State-of-the-art в компьютерном зрении. Batch Normalization, Training Tricks

## Batch Normalization
### Оригинальная статья [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/abs/1502.03167v3) [2015]

### Проблема

При обучении некоторых моделей в машинном обучении (например, логистической регрессии), мы нормализуем входные данные. Это делается для того, чтобы значения по всем признакам "лежали" в пространстве в районе начала координат в примерно одном масштабе.

![](./images/elongated_func_levels.gif)  
[Источник](https://towardsdatascience.com/types-of-optimization-algorithms-used-in-neural-networks-and-ways-to-optimize-gradient-95ae5d39529f)

То же самое можно делать и после каждого слоя (группы слоев) в нейронных сетях. Приведя данные к одному формату, мы обеспечиваем быструю и качественную сходимость при обучении (ускорение может достигать x10).

### Алгоритм Batch Normalization

![](./images/batch_norm.png)

### Обучение модели с использованием batch normalization:
1. Кроме параметров сети учим параметры $\gamma$ и $\beta$
2. При заморозке сети также замораживаем параметры $\gamma$ и $\beta$. Кроме этого прогоняем по тренировочной выборке, вычисляя средние по всем батчам $\mu_{B}$ и $\sigma_{B}$

![](./images/batch_norm_infer.png)

### Плюсы Batch Normalization
- **Позволяет установить выше learning rate** - выше скорость сходимости, решается проблема gradient explose
- **Работает как регуляризация** - каждый отдельный пример рассматривается в совокупности с другими примерами, что (засчет случайного сэмплирования батча) приводит к формированию совершенно нового примера

## [Bag of Tricks for Image Classification with Convolutional Neural Networks](https://arxiv.org/abs/1812.01187) [2018]

### Linear scaling learning rate
Увеличение batch size позволяет использовать силу параллелизма для обучения моделей быстрее. Однако при высоком batch size сходимость модели ухудшается.  
Поскольку при большем batch size уменьшается шум входных данных (ниже дисперсия), то можно безопасно увеличить learning rate.  

Лучшей практикой мастабирования learning rate в зависимости от размера батча оказалось линейное масштабирование: $0.1 * \frac{b}{256}$, где $0.1$ - начальный learning rate, $256$ - начальный batch size, $b$ - текущий batch size.

### Learning rate warmup
В начале обучения все параметры случайно проинициализированы. Соответственно, выбирать большой learning rate опасно - это приводит к численной нестабильности.  
Хорошей практикой является "нагревание", а именно линейное увеличение learning rate с некоторого маленького значения до желаемого.  

Например, при нагревании на $m$ батчах с желаемымым learning rate $\eta$, увеличение будет выполняться по закону $\frac{i}{m}\eta$, где $1\leq i \leq m$

### Zero $\gamma$ in Batch Normalization
Установить начальную $\gamma$ в batch normalization в 0. Оказывается полезной при сетках с residual блоками - таким образом блок не будет давать влияния вначале обучения, упрощая модель для оптимизации.

### Low-precision training

Модные GPU карты позволяют считать значительно быстрее, если перейти с float32 на float16. Так, Nvidia Tesla V100 в режиме FP32 выдает 14 TFLOPS, а в режиме FP16 выдает 100 TFLOPS (удалось достичь 2-3 раза ускорение при обучении ResNet-50).  
Статья [Mixed Precision Training](https://arxiv.org/abs/1710.03740) предлагает подход, при котором все вычисления происходят в FP16, а обновление весов в FP32 (для избегания ограничений с выходом за границы числа).

![](./images/mixed_precision_training.png)